# 🌱 Soil Classification using Deep Learning

## Overview
This notebook implements soil classification using Transfer Learning with MobileNetV2.

**Dataset:** 6 soil types (144 images)
- Alluvial soils
- Black soils  
- Chalky soils
- Clayey soils
- Loamy soils
- Red soils

**Improvements over baseline:**
- Transfer Learning with MobileNetV2
- Advanced data augmentation
- Comprehensive evaluation metrics
- Learning rate scheduling
- Better error handling

**Expected Performance:**
- Baseline CNN: ~81% accuracy
- Transfer Learning: ~92%+ accuracy

## 1. Import Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
from pathlib import Path
from PIL import Image

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, GlobalAveragePooling2D,
    Activation, Flatten, Dropout, Dense
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
)
from tensorflow.keras.utils import to_categorical

# Scikit-learn for metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_recall_fscore_support, roc_curve, auc
)
from sklearn.preprocessing import label_binarize

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 2. Configuration

Centralized configuration for easy experimentation

In [ ]:
# Configuration dictionary
CONFIG = {
    # Data settings
    'data_dir': r"C:\Users\fastf\Downloads\soil dataset\Soil Types",  # UPDATE THIS PATH
    'img_size': (150, 150),
    'batch_size': 32,
    'validation_split': 0.2,
    
    # Model settings
    'model_type': 'transfer_learning',  # 'cnn' or 'transfer_learning'
    'num_classes': 6,
    'base_model': 'MobileNetV2',  # MobileNetV2, EfficientNetB0, ResNet50
    
    # Training settings
    'epochs_stage1': 15,  # For transfer learning: train new layers
    'epochs_stage2': 20,  # For transfer learning: fine-tune all layers
    'epochs_cnn': 50,     # For baseline CNN
    'learning_rate': 0.001,
    'learning_rate_finetune': 0.0001,
    
    # Callbacks
    'early_stopping_patience': 10,
    'reduce_lr_patience': 3,
    
    # Paths
    'model_save_dir': './models',
    'logs_dir': './logs',
}

# Create directories
os.makedirs(CONFIG['model_save_dir'], exist_ok=True)
os.makedirs(CONFIG['logs_dir'], exist_ok=True)

print("Configuration loaded successfully!")
print(f"Model type: {CONFIG['model_type']}")
print(f"Image size: {CONFIG['img_size']}")
print(f"Batch size: {CONFIG['batch_size']}")

## 3. Data Loading and Exploration

In [ ]:
# Load dataset using TensorFlow's image_dataset_from_directory
print("Loading dataset...")

dataset = tf.keras.utils.image_dataset_from_directory(
    CONFIG['data_dir'],
    image_size=CONFIG['img_size'],
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    seed=42
)

# Get class names
class_labels = dataset.class_names
print(f"\nClass labels: {class_labels}")
print(f"Number of classes: {len(class_labels)}")

# Calculate dataset statistics
total_batches = len(dataset)
total_images = sum([len(batch[1]) for batch in dataset])
print(f"Total images: {total_images}")
print(f"Total batches: {total_batches}")

## 4. Data Visualization

In [ ]:
# Visualize sample images from each class
plt.figure(figsize=(15, 10))

# Get one batch
for images, labels in dataset.take(1):
    for i in range(min(12, len(images))):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_labels[labels[i]])
        plt.axis('off')

plt.tight_layout()
plt.savefig('sample_images.png', dpi=300, bbox_inches='tight')
plt.show()

print("Sample images saved as 'sample_images.png'")

## 5. Data Preparation

In [ ]:
# Split dataset into train and validation
train_size = int(0.8 * total_batches)
val_size = total_batches - train_size

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

print(f"Training batches: {train_size}")
print(f"Validation batches: {val_size}")

# Normalize the data (scale pixel values to [0, 1])
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

print("\nData normalization and optimization complete!")

## 6. Data Augmentation

Advanced augmentation to improve model generalization

In [ ]:
# Create data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

# Apply augmentation to training data
train_dataset_augmented = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=AUTOTUNE
)

print("Data augmentation configured successfully!")

# Visualize augmented images
plt.figure(figsize=(15, 8))
for images, labels in train_dataset.take(1):
    sample_image = images[0:1]  # Take first image
    
    for i in range(9):
        augmented_image = data_augmentation(sample_image, training=True)
        plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_image[0])
        plt.axis('off')
        plt.title(f"Augmentation {i+1}")

plt.suptitle('Data Augmentation Examples', fontsize=16)
plt.tight_layout()
plt.savefig('augmentation_examples.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Model Architecture

### Option A: Baseline CNN (Original)
### Option B: Transfer Learning with MobileNetV2 (Recommended)

In [ ]:
def build_baseline_cnn(input_shape=(150, 150, 3), num_classes=6):
    """
    Build baseline CNN model (original architecture)
    Expected accuracy: ~81%
    """
    model = Sequential([
        # First convolutional block
        Conv2D(32, (3, 3), input_shape=input_shape),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Second convolutional block
        Conv2D(32, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Third convolutional block
        Conv2D(64, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Flatten and fully connected layers
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

print("Baseline CNN architecture defined")

In [ ]:
def build_transfer_learning_model(input_shape=(150, 150, 3), num_classes=6):
    """
    Build Transfer Learning model with MobileNetV2
    Expected accuracy: ~92%+
    
    Why MobileNetV2?
    - Pre-trained on ImageNet (1.4M images)
    - Lightweight (14MB)
    - Fast inference
    - Excellent for small datasets
    """
    # Load pre-trained MobileNetV2 (without top classification layer)
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Build complete model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

print("Transfer Learning architecture defined")

In [ ]:
# Build model based on configuration
if CONFIG['model_type'] == 'transfer_learning':
    print("Building Transfer Learning model with MobileNetV2...")
    model, base_model = build_transfer_learning_model(
        input_shape=(*CONFIG['img_size'], 3),
        num_classes=CONFIG['num_classes']
    )
else:
    print("Building baseline CNN model...")
    model = build_baseline_cnn(
        input_shape=(*CONFIG['img_size'], 3),
        num_classes=CONFIG['num_classes']
    )
    base_model = None

# Compile model
model.compile(
    optimizer=Adam(learning_rate=CONFIG['learning_rate']),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

# Count parameters
total_params = model.count_params()
print(f"\nTotal parameters: {total_params:,}")

## 8. Training Callbacks

In [ ]:
import datetime

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=CONFIG['early_stopping_patience'],
    restore_best_weights=True,
    verbose=1
)

# Learning rate reduction
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=CONFIG['reduce_lr_patience'],
    min_lr=1e-7,
    verbose=1
)

# Model checkpoint
checkpoint = ModelCheckpoint(
    filepath=os.path.join(CONFIG['model_save_dir'], 'best_model_{epoch:02d}_{val_accuracy:.4f}.h5'),
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# TensorBoard
log_dir = os.path.join(CONFIG['logs_dir'], datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'
)

callbacks = [early_stopping, reduce_lr, checkpoint, tensorboard_callback]

print("Callbacks configured successfully!")
print(f"TensorBoard logs will be saved to: {log_dir}")
print("To view: tensorboard --logdir=./logs")

## 9. Model Training

### Two-Stage Training for Transfer Learning:
1. **Stage 1**: Train only new layers (frozen base)
2. **Stage 2**: Fine-tune entire model (unfrozen base)

In [ ]:
if CONFIG['model_type'] == 'transfer_learning':
    print("="*70)
    print("STAGE 1: Training new layers only (base model frozen)")
    print("="*70)
    
    history_stage1 = model.fit(
        train_dataset_augmented,
        validation_data=val_dataset,
        epochs=CONFIG['epochs_stage1'],
        callbacks=callbacks,
        verbose=1
    )
    
    print("\nStage 1 training complete!")
    print(f"Best validation accuracy: {max(history_stage1.history['val_accuracy']):.4f}")
else:
    print("="*70)
    print("Training baseline CNN model")
    print("="*70)
    
    history = model.fit(
        train_dataset_augmented,
        validation_data=val_dataset,
        epochs=CONFIG['epochs_cnn'],
        callbacks=callbacks,
        verbose=1
    )
    
    print("\nTraining complete!")
    print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")

In [ ]:
if CONFIG['model_type'] == 'transfer_learning':
    print("\n" + "="*70)
    print("STAGE 2: Fine-tuning entire model (base model unfrozen)")
    print("="*70)
    
    # Unfreeze base model
    base_model.trainable = True
    print(f"Base model layers now trainable: {len(base_model.layers)}")
    
    # Recompile with lower learning rate (CRITICAL!)
    model.compile(
        optimizer=Adam(learning_rate=CONFIG['learning_rate_finetune']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"Learning rate reduced to: {CONFIG['learning_rate_finetune']}")
    
    # Continue training
    history_stage2 = model.fit(
        train_dataset_augmented,
        validation_data=val_dataset,
        epochs=CONFIG['epochs_stage2'],
        callbacks=callbacks,
        initial_epoch=len(history_stage1.history['loss']),
        verbose=1
    )
    
    print("\nStage 2 training complete!")
    print(f"Best validation accuracy: {max(history_stage2.history['val_accuracy']):.4f}")
    
    # Combine histories
    history = type('obj', (object,), {
        'history': {
            'loss': history_stage1.history['loss'] + history_stage2.history['loss'],
            'accuracy': history_stage1.history['accuracy'] + history_stage2.history['accuracy'],
            'val_loss': history_stage1.history['val_loss'] + history_stage2.history['val_loss'],
            'val_accuracy': history_stage1.history['val_accuracy'] + history_stage2.history['val_accuracy']
        }
    })()

## 10. Training Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Add vertical line for stage transition if transfer learning
if CONFIG['model_type'] == 'transfer_learning':
    stage1_epochs = len(history_stage1.history['loss'])
    for ax in axes:
        ax.axvline(x=stage1_epochs-1, color='red', linestyle='--', 
                   label='Fine-tuning starts', linewidth=2)
        ax.legend()

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Print final metrics
print("\n" + "="*50)
print("FINAL TRAINING METRICS")
print("="*50)
print(f"Final Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Val Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Best Val Accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"Final Train Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Val Loss: {history.history['val_loss'][-1]:.4f}")
print("="*50)

## 11. Comprehensive Model Evaluation

In [ ]:
# Collect predictions
print("Collecting predictions on validation set...")
y_true = []
y_pred = []
y_pred_proba = []

for images, labels in val_dataset:
    predictions = model.predict(images, verbose=0)
    y_pred_proba.extend(predictions)
    y_pred.extend(np.argmax(predictions, axis=1))
    y_true.extend(labels.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)

print(f"Collected {len(y_true)} predictions")

In [ ]:
# Classification Report
print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(classification_report(y_true, y_pred, target_names=class_labels, digits=4))

# Per-class metrics
precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, average=None
)

metrics_df = pd.DataFrame({
    'Class': class_labels,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("\nPer-Class Metrics:")
print(metrics_df.to_string(index=False))

# Save to CSV
metrics_df.to_csv('classification_metrics.csv', index=False)
print("\nMetrics saved to 'classification_metrics.csv'")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate accuracy from confusion matrix
accuracy = np.trace(cm) / np.sum(cm)
print(f"\nOverall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

In [ ]:
# ROC Curves for multi-class classification
y_true_bin = label_binarize(y_true, classes=range(len(class_labels)))

plt.figure(figsize=(12, 8))

for i, label in enumerate(class_labels):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, linewidth=2, label=f'{label} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random (AUC = 0.500)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Multi-class Classification', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate macro-average AUC
all_fpr = np.unique(np.concatenate([roc_curve(y_true_bin[:, i], y_pred_proba[:, i])[0] 
                                     for i in range(len(class_labels))]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(len(class_labels)):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
    mean_tpr += np.interp(all_fpr, fpr, tpr)
mean_tpr /= len(class_labels)
macro_auc = auc(all_fpr, mean_tpr)

print(f"\nMacro-average AUC: {macro_auc:.4f}")

## 12. Save Model with Metadata

In [ ]:
import json

# Save final model
model_path = os.path.join(CONFIG['model_save_dir'], 'soil_classifier_final.h5')
model.save(model_path)
print(f"Model saved to: {model_path}")

# Save model metadata
metadata = {
    'model_type': CONFIG['model_type'],
    'base_model': CONFIG['base_model'] if CONFIG['model_type'] == 'transfer_learning' else 'CNN',
    'num_classes': CONFIG['num_classes'],
    'class_labels': class_labels,
    'img_size': CONFIG['img_size'],
    'training_date': datetime.datetime.now().isoformat(),
    'total_parameters': int(total_params),
    'performance': {
        'final_train_accuracy': float(history.history['accuracy'][-1]),
        'final_val_accuracy': float(history.history['val_accuracy'][-1]),
        'best_val_accuracy': float(max(history.history['val_accuracy'])),
        'final_train_loss': float(history.history['loss'][-1]),
        'final_val_loss': float(history.history['val_loss'][-1]),
        'macro_auc': float(macro_auc)
    },
    'hyperparameters': {
        'batch_size': CONFIG['batch_size'],
        'learning_rate': CONFIG['learning_rate'],
        'learning_rate_finetune': CONFIG['learning_rate_finetune'] if CONFIG['model_type'] == 'transfer_learning' else None,
        'total_epochs': len(history.history['loss'])
    }
}

metadata_path = model_path.replace('.h5', '_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata saved to: {metadata_path}")
print("\n" + "="*50)
print("MODEL SUMMARY")
print("="*50)
print(json.dumps(metadata, indent=2))
print("="*50)

## 13. Prediction Function with Error Handling

In [ ]:
def validate_image(img_path):
    """
    Validate image file before processing
    """
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Image not found: {img_path}")
    
    allowed_extensions = ['.png', '.jpg', '.jpeg', '.bmp']
    ext = os.path.splitext(img_path)[1].lower()
    if ext not in allowed_extensions:
        raise ValueError(f"Invalid file type: {ext}. Allowed: {allowed_extensions}")
    
    try:
        img = Image.open(img_path)
        img.verify()
        return True
    except Exception as e:
        raise ValueError(f"Corrupt or invalid image file: {e}")


def predict_soil_type(img_path, model, class_labels, img_size=(150, 150)):
    """
    Predict soil type from image with comprehensive error handling
    
    Args:
        img_path: Path to soil image
        model: Trained Keras model
        class_labels: List of class names
        img_size: Target image size (width, height)
    
    Returns:
        Dictionary with prediction results or error information
    """
    try:
        # Validate input
        validate_image(img_path)
        
        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array / 255.0  # Normalize
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        
        # Predict
        predictions = model.predict(img_array, verbose=0)
        pred_class = np.argmax(predictions[0])
        confidence = predictions[0][pred_class]
        
        # Prepare result
        result = {
            'success': True,
            'predicted_class': class_labels[pred_class],
            'confidence': float(confidence),
            'all_probabilities': {
                class_labels[i]: float(predictions[0][i])
                for i in range(len(class_labels))
            }
        }
        
        return result
    
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

print("Prediction function defined successfully!")

## 14. Test Prediction

**Note:** Update the image path below to test prediction on your own images

In [ ]:
# Example prediction (update path to your test image)
test_image_path = r"C:\Users\fastf\Downloads\soil dataset\Soil Types\Clayey soils\clay 9.png"

if os.path.exists(test_image_path):
    # Make prediction
    result = predict_soil_type(
        test_image_path,
        model,
        class_labels,
        img_size=CONFIG['img_size']
    )
    
    # Display result
    if result['success']:
        print("\n" + "="*50)
        print("PREDICTION RESULT")
        print("="*50)
        print(f"Predicted Soil Type: {result['predicted_class']}")
        print(f"Confidence: {result['confidence']:.4f} ({result['confidence']*100:.2f}%)")
        print("\nAll Class Probabilities:")
        for soil_type, prob in sorted(result['all_probabilities'].items(), 
                                     key=lambda x: x[1], reverse=True):
            print(f"  {soil_type}: {prob:.4f} ({prob*100:.2f}%)")
        print("="*50)
        
        # Visualize prediction
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Show image
        img = Image.open(test_image_path)
        ax1.imshow(img)
        ax1.axis('off')
        ax1.set_title(f'Predicted: {result["predicted_class"]}\nConfidence: {result["confidence"]:.2%}',
                     fontsize=12, fontweight='bold')
        
        # Show probability distribution
        probs = result['all_probabilities']
        labels = list(probs.keys())
        values = list(probs.values())
        
        bars = ax2.barh(labels, values, color='skyblue')
        bars[labels.index(result['predicted_class'])].set_color('green')
        ax2.set_xlabel('Probability', fontsize=10)
        ax2.set_title('Class Probabilities', fontsize=12, fontweight='bold')
        ax2.set_xlim([0, 1])
        
        plt.tight_layout()
        plt.savefig('prediction_result.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print(f"\nPrediction failed: {result['error']}")
else:
    print(f"Test image not found: {test_image_path}")
    print("Please update the path to test prediction")

## 15. Final Summary & Next Steps

In [ ]:
print("\n" + "="*70)
print("FINAL SUMMARY")
print("="*70)
print(f"\nModel Type: {CONFIG['model_type']}")
if CONFIG['model_type'] == 'transfer_learning':
    print(f"Base Model: {CONFIG['base_model']}")
print(f"\nDataset Size: {total_images} images")
print(f"Number of Classes: {CONFIG['num_classes']}")
print(f"Classes: {', '.join(class_labels)}")
print(f"\nTotal Parameters: {total_params:,}")
print(f"Image Size: {CONFIG['img_size']}")
print(f"Batch Size: {CONFIG['batch_size']}")

print("\n" + "-"*70)
print("PERFORMANCE METRICS")
print("-"*70)
print(f"Best Validation Accuracy: {max(history.history['val_accuracy']):.4f} ({max(history.history['val_accuracy'])*100:.2f}%)")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f} ({history.history['val_accuracy'][-1]*100:.2f}%)")
print(f"Macro-average AUC: {macro_auc:.4f}")

print("\n" + "-"*70)
print("FILES GENERATED")
print("-"*70)
print("✓ sample_images.png - Dataset visualization")
print("✓ augmentation_examples.png - Data augmentation examples")
print("✓ training_history.png - Training curves")
print("✓ confusion_matrix.png - Confusion matrix heatmap")
print("✓ roc_curves.png - ROC curves for all classes")
print("✓ classification_metrics.csv - Per-class metrics")
print("✓ prediction_result.png - Example prediction visualization")
print(f"✓ {model_path} - Saved model")
print(f"✓ {metadata_path} - Model metadata")

print("\n" + "-"*70)
print("NEXT STEPS")
print("-"*70)
print("1. Try different base models (EfficientNetB0, ResNet50)")
print("2. Experiment with image size (224x224)")
print("3. Collect more training data")
print("4. Implement k-fold cross-validation")
print("5. Deploy model as REST API")
print("6. Add model interpretability (Grad-CAM)")
print("7. Optimize for mobile deployment (TFLite)")

print("\n" + "="*70)
print("Training complete! 🎉")
print("="*70)

## 16. How to Load and Use Saved Model

In [ ]:
# Example: Load saved model and make predictions

# Load model
loaded_model = load_model(model_path)
print("Model loaded successfully!")

# Load metadata
with open(metadata_path, 'r') as f:
    loaded_metadata = json.load(f)

print("\nModel Metadata:")
print(f"Model Type: {loaded_metadata['model_type']}")
print(f"Classes: {loaded_metadata['class_labels']}")
print(f"Best Accuracy: {loaded_metadata['performance']['best_val_accuracy']:.4f}")

# Make prediction with loaded model
# result = predict_soil_type(
#     'path/to/new/image.jpg',
#     loaded_model,
#     loaded_metadata['class_labels'],
#     tuple(loaded_metadata['img_size'])
# )
# print(result)